In [1]:
!nvidia-smi

Sun Dec 13 18:15:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64       Driver Version: 440.64       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:03:00.0 Off |                  N/A |
| 42%   60C    P2    78W / 280W |   3954MiB / 24219MiB |      3%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           Off  | 00000000:04:00.0 Off |                  N/A |
| 41%   38C    P2    53W / 280W |   5586MiB / 24220MiB |     25%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [2]:
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
import torchvision.datasets
from torchvision import transforms

In [3]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [4]:
#PATH = 'CIFAR10_VGG16_compressed.pth'
PATH = 'CIFAR10_VGG16_compressed_v2.pth'
model = torch.load(PATH)
model.eval()

VGG(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 2, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): Conv2d(2, 14, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (2): Conv2d(14, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (1): ReLU(inplace=True)
    (2): Sequential(
      (0): Conv2d(64, 27, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): Conv2d(27, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (2): Conv2d(24, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Sequential(
      (0): Conv2d(64, 26, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): Conv2d(26, 27, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (2): Conv2d(27, 128, kernel_size=(1, 1), stride=(1, 1))
    )
    (6): ReLU(inplace=True)
    (7): Sequential(
      (0): Conv2d(128, 24, kerne

In [5]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:1


In [6]:
for param in model.parameters():
    param.requires_grad = True

In [7]:
model = model.to(device)

In [8]:
#также будем использовать аугментацию: рандомные повороты на 7 градусов и отражение относительно вертикали.

#преобразование трейна
train_tf = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(7),
    transforms.ToTensor()
])

#тест не меняем
test_tf = transforms.Compose([
    transforms.ToTensor()
])

train = torchvision.datasets.CIFAR10('./', download=True, train=True, transform=train_tf)
test = torchvision.datasets.CIFAR10('./', download=True, train=False, transform=test_tf)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:1


In [10]:
# data loader
trainloader = torch.utils.data.DataLoader(train, batch_size = 64, shuffle = True)
testloader = torch.utils.data.DataLoader(test, batch_size = 256, shuffle = False)

In [11]:
#В качестве лоса возмем кросс-энтропию. Оптимизатор - Адам
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

#добавим уменьшение лернинг рейта, если выходим на плато. Это решение будем принимать по валидационной выборке.
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=7, factor=0.2)

In [12]:
test_acc = 0.0
len_test = 0.0

for _, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs.to(device), targets.to(device)
    outputs = model(inputs)
    accuracy_add = (outputs.argmax(dim=1) == targets).float().sum().data.cpu()
    test_acc += accuracy_add
    len_test += len(targets)
    
test_acc = test_acc / len_test
test_acc = np.round(test_acc.numpy(), 4)

In [13]:
print('Accuracy on test after compression:', test_acc)

Accuracy on test after compression: 0.1496


In [14]:
def train(epoch):
    running_loss = 0.0
    train_acc = 0.0
    test_acc = 0.0
    len_train = 0.0
    len_test = 0.0
    
    loss_train = 0.0
  
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss_val = loss(outputs, targets)
        loss_val.backward()
        optimizer.step()
        accuracy_add = (outputs.argmax(dim=1) == targets).float().sum().data.cpu()
        train_acc += accuracy_add
        len_train += len(targets)
        loss_train += len(targets) * loss_val.item()
        running_loss += loss_val.item()
        
    for _, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        accuracy_add = (outputs.argmax(dim=1) == targets).float().sum().data.cpu()
        test_acc += accuracy_add
        len_test += len(targets)

        
    lr_scheduler.step(running_loss)

    return train_acc / len_train, test_acc / len_test

In [15]:
accuracy_history_test = []
accuracy_history_train = []

for epoch in range(0, 19):
    train_acc, test_acc = train(epoch)
    accuracy_history_test.append(test_acc)
    accuracy_history_train.append(train_acc)
    acc_train = np.round(train_acc.numpy(), 4)
    acc_test = np.round(test_acc.numpy(), 4)
    print('Epoch:', epoch, '   acc_train:', acc_train, '   test_acc:', acc_test)
    if acc_test > 0.8: break

Epoch: 0    acc_train: 0.6794    test_acc: 0.6949
Epoch: 1    acc_train: 0.7293    test_acc: 0.739
Epoch: 2    acc_train: 0.7485    test_acc: 0.7412
Epoch: 3    acc_train: 0.7663    test_acc: 0.7819
Epoch: 4    acc_train: 0.7629    test_acc: 0.7681
Epoch: 5    acc_train: 0.7714    test_acc: 0.7728
Epoch: 6    acc_train: 0.7913    test_acc: 0.7945
Epoch: 7    acc_train: 0.7994    test_acc: 0.7856
Epoch: 8    acc_train: 0.8006    test_acc: 0.7933
Epoch: 9    acc_train: 0.7936    test_acc: 0.7714
Epoch: 10    acc_train: 0.8049    test_acc: 0.8045


In [16]:
def count_parameters(model): 
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [20]:
print('Num params in model after compression:', count_parameters(model))
print('Num params in model before compression:', 138357544)
print('Ratio:', 138357544 / count_parameters(model))

Num params in model after compression: 125549659
Num params in model before compression: 138357544
Ratio: 1.1020144945196546
